# Tech Intro

## Main tables

In [1]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [2]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = real_estate_list[10] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [3]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

22/11/06 18:47:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/06 18:47:40 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/11/06 18:47:41 WARN FairSchedulableBuilder: Fair Scheduler configuration file not found so jobs will be scheduled in FIFO order. To use fair scheduling, configure pools in fairscheduler.xml or set spark.scheduler.allocation.file to a file that contains the configuration.
2022-11-06 18:47:41,374:35770(0x7f791c788700):ZOO_INFO@log_env@753: Client environment:zookeeper.version=zookeeper C client 3.4.8
2022-11-06 18:47:41,374:35770(0x7f791c788700)

## Read the spec-file from Hadoop and show some data sample

In [24]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10000, False)

+-----------------+------------+-----------+------------+-------------+-------+--------+-------------+------------+-----------+------+
|ag_object_type_id|ag_object_id|type       |places_count|price_monthly|is_free|currency|location_type|purpose_type|price_entry|id    |
+-----------------+------------+-----------+------------+-------------+-------+--------+-------------+------------+-----------+------+
|null             |25996       |null       |null        |null         |null   |rur     |null         |null        |null       |10    |
|null             |17611       |null       |null        |null         |null   |rur     |null         |null        |null       |1000  |
|null             |45464       |null       |null        |null         |null   |rur     |null         |null        |null       |10000 |
|null             |146432      |null       |null        |null         |null   |rur     |null         |null        |null       |100002|
|null             |38671       |null       |null       

## Show columns' properties

In [5]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- ag_object_type_id: string (nullable = true)
 |-- ag_object_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- places_count: string (nullable = true)
 |-- price_monthly: string (nullable = true)
 |-- is_free: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- location_type: string (nullable = true)
 |-- purpose_type: string (nullable = true)
 |-- price_entry: string (nullable = true)
 |-- id: string (nullable = true)



## Total rows (count)

In [6]:
eda_df.count()

241576

## Simple func to execute and show the result of the SQL-instruction

In [7]:
def execute(select_instruction: str = "select count(*) from realEstate") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from realEstate"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show()

## Create the temp-table-view from the DF (called as "eda_sql")

In [8]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [33]:
execute("select count(*) from eda_sql")
#execute("select * from eda_sql where price_monthly > 0")

+--------+
|count(1)|
+--------+
|  241576|
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [10]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,ag_object_type_id,ag_object_id,type,places_count,price_monthly,is_free,currency,location_type,purpose_type,price_entry,id
0,241576,0,170337,241355,241534,240878,0,240049,241365,241556,0


In [11]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,ag_object_type_id,ag_object_id,type,places_count,price_monthly,is_free,currency,location_type,purpose_type,price_entry,id
0,0,0,0,0,31,19,0,0,0,20,0


In [12]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,ag_object_type_id,ag_object_id,type,places_count,price_monthly,is_free,currency,location_type,purpose_type,price_entry,id
0,0,0,2615,0,0,0,0,0,0,0,0


In [18]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
ag_object_type_id,241576,0,0
ag_object_id,0,0,0
type,170337,0,2615
places_count,241355,0,0
price_monthly,241534,31,0
is_free,240878,19,0
currency,0,0,0
location_type,240049,0,0
purpose_type,241365,0,0
price_entry,241556,20,0


In [19]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [20]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
ag_object_type_id,100.000,0.000,0.000
ag_object_id,0.000,0.000,0.000
type,70.511,0.000,1.082
places_count,99.909,0.000,0.000
price_monthly,99.983,0.013,0.000
is_free,99.711,0.008,0.000
currency,0.000,0.000,0.000
location_type,99.368,0.000,0.000
purpose_type,99.913,0.000,0.000
price_entry,99.992,0.008,0.000


### SQL-instructions

In [73]:
#Find dupes or misunderstanding or useless static values
for col in eda_df.columns: 
    execute("select count(distinct("+ col + ")) from eda_sql")
    
        
#Check all the 'price-attrs'
execute("SELECT price_monthly\
        FROM eda_sql\
        WHERE price_monthly IS NOT NULL\
            AND price_monthly != 0\
        ORDER BY price_monthly ASC")    
execute("SELECT price_entry\
        FROM eda_sql\
        WHERE price_entry IS NOT NULL\
            AND price_entry != 0\
        ORDER BY price_entry ASC")



+---------------------------------+
|count(DISTINCT ag_object_type_id)|
+---------------------------------+
|                                0|
+---------------------------------+

+----------------------------+
|count(DISTINCT ag_object_id)|
+----------------------------+
|                      241555|
+----------------------------+

+--------------------+
|count(DISTINCT type)|
+--------------------+
|                   6|
+--------------------+

+----------------------------+
|count(DISTINCT places_count)|
+----------------------------+
|                          29|
+----------------------------+

+-----------------------------+
|count(DISTINCT price_monthly)|
+-----------------------------+
|                            6|
+-----------------------------+

+-----------------------+
|count(DISTINCT is_free)|
+-----------------------+
|                      2|
+-----------------------+

+------------------------+
|count(DISTINCT currency)|
+------------------------+
|                 

## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [ ]:
spark.stop()